In [69]:
import numpy as np
import pandas as pd
import os
import tqdm
import itertools
from collections import Counter

import sys
sys.path.append('../')

from util.text_preprocessing import tokenize_okt

# Load Data

## 전체 코멘트 불러오기

In [70]:
total_comments_df = pd.read_pickle("dataset/total_raw_token/0710 comments_with_okt-tokenized.pkl")
total_comments_df = total_comments_df.loc[:,['id','Sentence','Emotion', 'comment_token']]
total_comments_df.head(5)


,id,Sentence,Emotion,comment_token
0,0,청소 네가 대신 해 줘,중립,"[청소, 네, 대신, 해, 주다]"
1,1,둘 다 청소 하기 싫어 귀찮아,중립,"[둘, 청소, 싫다, 귀찮다]"
2,2,둘 다 하기 싫어서 화내,분노,"[둘, 싫다, 화내]"
3,3,그럼 방세는 어떡해,슬픔,"[그렇다, 방, 세다, 어떻다, 해]"
4,4,권택인 줄 알았는데 그런 사람이 생겼나 보더라고,슬픔,"[권, 택인, 줄, 알다, 그렇다, 사람, 생기다, 보더]"


## 감정 사전 만들기
- 6 emotion : happy, sad, disgust, angry, surprised, fear
- 홍종선, 정연주. (2009). 감정동사의 범주 규정과 유형 분류. 한국어학, 45(), 387-420.

In [71]:
six_emodict_df = pd.read_excel("자료/six_emotion_bigram2_0403.xlsx")
six_emodict_df.head()
#엑셀 받아올 때 encoding = 'utf-8' 생략해도 object형식이여서 괜찮을 듯

,happy,sad,disgust,angry,surprised,fear
0,가뿐하다,가슴앓이,가소롭다,갈기갈기,갑작스럽다,가혹하다
1,감개무량하다,가엽다,거북하다,개새끼,경악하다,강압적
2,감격스럽다,가엾다,경박하다,개자식,경이,겁쟁이
3,감격하다,가혹하다,괴상하다,격노하다,급작스럽다,공포감
4,감동스럽다,각박하다,괴팍하다,격분하다,기겁하다,공포스럽다


In [72]:
# 감정 별 단어 개수
six_emodict_df.count()


happy        113
sad          174
disgust       77
angry        110
surprised     47
fear          38
dtype: int64

In [73]:
# 감정 리스트에 변수 할당 
six_emotion = six_emodict_df.columns.tolist()
print(six_emotion)
 

for emo in six_emotion:
    # 단어 사전에서 emo_words 받아오기
    emo_words = six_emodict_df[emo].dropna().tolist() 
    # 단어 토큰화, 토큰화 후 nan값 된 단어 제거
    emo_words = [tokenize_okt(x)[0] for x in emo_words if len(tokenize_okt(x))>0]
    # 중복 제거
    emo_words = sorted(list(set(emo_words)))
    # 각 감정변수에 차례대로 할당
    globals()[emo] = emo_words
    
sad[:5]


['happy', 'sad', 'disgust', 'angry', 'surprised', 'fear']


['가슴앓이', '가엽다', '가엾다', '가혹하다', '각박하다']

In [74]:
bi_happy = [('속','시원하다'),('낯','간지럽다'),('정','들다'),('홀','가분')]

bi_sad = [('가슴','아프다'),('기운','없다'),('남','부럽다'),('뒤','숭숭'),('맥','빠지다'),
              ('복','받치다'),('뼈','아프다'),('뼈','저리다'),('애','끓다'),('애','닮다'),
              ('애','타다'),('처','연하다'),('속','썩다'),('속','앓이'),('속','타다')]

bi_disgust =  [('못','밉다'),('정','떨어지다'),('못','마땅하다'),('시','덥다'),('남','부끄럽다'),
                   ('징','그렇다'),('뼈','저리다'),('애','끓다'),('애','닮다'),('처','연하다'),
                   ('속','썩다'),('속','앓이'),('속','타다')]

bi_angry = [('못','되다'),('삐','딱하다'),('비','딱하다'),('약','오르다'),('치','떨리다'),
                ('치밀','오르다'),('성','나다'),('못','마땅하다'),('정','떨어지다'),('남','부끄럽다')]

bi_surprised = [('기상','천외'),('얼','떨다')]

bi_fear = [('뒤','숭숭')]

# 코멘트 데이터에 감정 레이블 달기 

In [75]:
test = ["가슴 아프다 정말", #- sad
        "요즘 상황이 뒤숭숭하다",#sad fear
        "어어어어엉 우웅우 감동스럽다",#happy
       "낯간지러워"]
#위 bi_감정에 할당한 단어들로 바꿔도 정상적으로 돌아감, bigram은 두 개의 단어가 합쳐져 감정이 되는 것
        
test_token = []
for i in test:
    test_token.append(tokenize_okt(i))
print(test_token)

[['가슴', '아프다', '정말'], ['요즘', '상황', '뒤', '숭숭'], ['어어', '엉', '우웅', '우', '감동', '스럽다'], ['낯', '간지럽다']]


## 댓글에 특정 감정에 해당하는 단어가 등장하면 update

- bigram
- unigram

In [76]:
total_comments_df['comment_token'][:10]

0                   [청소, 네, 대신, 해, 주다]
1                     [둘, 청소, 싫다, 귀찮다]
2                          [둘, 싫다, 화내]
3                 [그렇다, 방, 세다, 어떻다, 해]
4     [권, 택인, 줄, 알다, 그렇다, 사람, 생기다, 보더]
5                         [그냥, 걷다, 있다]
6                           [고등학교, 동창]
7    [처음, 학원, 만나다, 서로, 좋아하다, 사귀다, 되어다]
8             [내, 애정, 표현, 자다, 자주, 싸우다]
9                           [오늘, 헤어지다]
Name: comment_token, dtype: object

In [81]:
emo_set = [happy, sad, disgust, angry, surprised, fear]
emo_bigram_set = [bi_happy, bi_sad, bi_disgust, bi_angry, bi_surprised, bi_fear]

check_token_lst = []
emo_cnt_lst = []

total_num = len(total_comments_df)

# comment 돌기
for i,Sentence in enumerate(total_comments_df['comment_token']):
    print(i/total_num,end="\r")
    check_token = []
    emo_cnt = np.zeros(len(emo_set))
    
    # bigram 처리
    for j, biwords in enumerate(emo_bigram_set):
        for a, b in biwords:
             if (a in Sentence) & (b in Sentence):
                check_token.append(a+b)
                emo_cnt[j] += 1 

    # unigram 처리
    for token in Sentence:
        for k, words in enumerate(emo_set):
            if token in words:
                emo_cnt[k] += 1
                check_token.append(token)

    check_token_lst.append(check_token)
    emo_cnt_lst.append(emo_cnt)

0.019210774669289953

0.03894940576163283

0.07180446918438185

0.09127117424060656

0.11161764000191948

...

0.9241966185197626

0.941375945742758

0.9560599516931395

0.9771262216677064

0.9942415662939681

In [88]:
# 비율로 환산
sum_of_emo = np.array(emo_cnt_lst).sum(axis=1,keepdims=True)
#print(sum_of_emo)

#print(np.array(emo_cnt_lst))

emo_ratio = np.array(emo_cnt_lst) / sum_of_emo
#print(emo_ratio)

emo_ratio[np.isnan(emo_ratio)] = 0

emo_df = pd.DataFrame(emo_ratio,columns = ['happy','sad','disgust','angry','surprised','fear'])

<ipython-input-88-6d600ab429fd>:7: RuntimeWarning: invalid value encountered in true_divide
  emo_ratio = np.array(emo_cnt_lst) / sum_of_emo


In [90]:
comment_with_emo = pd.concat([emo_df,
                              total_comments_df,
                              pd.DataFrame({"check":pd.Series(check_token_lst)})],
                             sort=False,axis=1)
# 최소 1개의 감정단어 등장한 댓글만 뽑기
comment_with_emo = comment_with_emo[comment_with_emo['check'].apply(len) > 0]
print(comment_with_emo.shape)
comment_with_emo.head()

(10406, 11)


,happy,sad,disgust,angry,surprised,fear,id,Sentence,Emotion,comment_token,check
0,0.0,0.0,0.0,1.0,0.0,0.0,0,청소 네가 대신 해 줘,중립,"[청소, 네, 대신, 해, 주다]",[네]
1,0.0,0.0,0.5,0.5,0.0,0.0,1,둘 다 청소 하기 싫어 귀찮아,중립,"[둘, 청소, 싫다, 귀찮다]","[싫다, 싫다]"
2,0.0,0.0,0.5,0.5,0.0,0.0,2,둘 다 하기 싫어서 화내,분노,"[둘, 싫다, 화내]","[싫다, 싫다]"
13,0.0,0.0,0.0,0.0,0.0,1.0,13,부모님한테 아직 말 안 했는데 말하기가 두려워,중립,"[부모님, 아직, 말, 안, 말, 두렵다]",[두렵다]
17,0.0,1.0,0.0,0.0,0.0,0.0,17,부모님도 다 슬퍼하지,슬픔,"[부모님, 슬프다]",[슬프다]


In [91]:
print("{}개의 댓글 중 {} 개의 댓글이 감정 단어를 보유.".format(len(total_comments_df),len(comment_with_emo)))

62517개의 댓글 중 10406 개의 댓글이 감정 단어를 보유.


In [92]:
comment_with_emo['check'].apply(len).max()

75

In [93]:
comment_with_emo[comment_with_emo['check'].apply(len)==109]

,happy,sad,disgust,angry,surprised,fear,id,Sentence,Emotion,comment_token,check


## 감정 단어 보유 비율에 따라 labeling

In [96]:
def label_with_major_emotion(df, ratio):
    over_index = np.where((df.happy > ratio) |
                          (df.sad > ratio) |
                          (df.disgust > ratio) |
                          (df.angry > ratio) |
                          (df.surprised > ratio) |
                          (df.fear > ratio))[0]

    over_df = df.iloc[over_index.tolist(),:]

    # Skipna = True will skip all the Na values 
    # find maximum along column axis 
    over_df['emotion'] = over_df.iloc[:,:6].idxmax(axis = 1, skipna = True)
    print(Counter(over_df.emotion))
    
    return over_df

In [97]:
label_with_major_emotion(comment_with_emo, 0.5).to_pickle("dataset/model/0710 comment_with_emo_over0.5.pkl")

Counter({'sad': 3091, 'angry': 1935, 'happy': 1616, 'fear': 903, 'surprised': 579, 'disgust': 495})


<ipython-input-96-e4142b41e118>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  over_df['emotion'] = over_df.iloc[:,:6].idxmax(axis = 1, skipna = True)


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/model/0710 comment_with_emo_over0.5.pkl'